# `crunchy-snow` Background
Quinn Brencher, University of Washington, 08/16/2024

The `crunchy-snow` project is an effort to predict snow depth from spaceborne remote sensing data using machine learning models. This notebook provides motivation, theoretical background in remote sensing and convolutional neural networks, and a brief description of the dataset preparation methods. 

## Project Motivation

### Why measure snow depth?
"Runoff from seasonal snow provides water for billions of people (Barnett et al., 2005; Mankin et al., 2015), supplies up to 70% of the annual discharge in the western United States (WUS; Li et al. (2017)), generates clean hydroelectric power, and supports agricultural and recreation industries at a total value estimated in the trillions of dollars (Sturm et al., 2017). Understanding the spatial distribution of snow water equivalent (SWE), the defining hydrologic variable of the seasonal snowpack, is essential for effective management of this critical resource (Bales et al., 2006). SWE is the product of snow depth and snow density relative to water, with snow depth spatial variability providing the majority of the variation in SWE values (Sturm et al., 2010). Therefore, accurate measurements of snow depth are crucial for global SWE estimation, since measurement of snow depth is typically much easier and lower cost than direct measurements of SWE.

Current operational snow depth measurement techniques lack either the spatial or temporal resolution necessary to accurately monitor basin-scale snow depth patterns for a variety of scientific and resource management applications (NASEM, 2018)." -- Hoppinen et al. (2024)

### What spaceborne remote sensing data can provide information about snow depth? 
In this project (so far) we consider two primary data sources that can provide information about snow depth from space. 

The first of these is [Sentinel-1](https://planetarycomputer.microsoft.com/dataset/sentinel-1-rtc) C-band synthetic aperture radar (SAR) data. "SAR is a type of active microwave remote sensing that can operate in all weather conditions, does not rely on solar illumination, and is capable of producing datasets at meter-scale spatial resolution from spaceborne platforms. Unlike optical and lidar techniques, SAR signals penetrate the snow surface and interact with the snowpack, allowing for measurements of snowpack properties. The extent of this penetration and which snowpack features are interacted with varies depending on the SAR signal’s frequency (Rosen et al., 2000; Tsai et al., 2019; Marshall et al., 2021). Thus, SAR methods for retrievals of snow depth and SWE have the potential to meet the National Academies of Science (2018) Decadal Survey requirement of snow depth and SWE measurements at 100 m spatial resolution." -- Hoppinen et al. (2024)

The second data source is [Sentinel-2](https://planetarycomputer.microsoft.com/dataset/sentinel-2-l2a) imagery. While imagery does not provide direct information about snow depth, as sunlight does not penetrate deep into the snowpack, it does provide valuable contextual information including snow extent, snowline elevation, tree cover, snow color, and whether specific terrain features are buried or exposed. 

We can also make use of several auxilary datasets, including [elevation](https://planetarycomputer.microsoft.com/dataset/cop-dem-glo-30), [forest cover fraction](https://earth.esa.int/eogateway/documents/20142/1366722/Sterckx-et-al-The-PROBA-V-mission-image-processing-and-calibration-JRS-2014.pdf), latitude and longitude, and day of water year. 

### Previous work
Recent work has demonstrated algorithms with the potential to retrieve snow-depth measurements from Sentinel-1 synthetic aperture radar (SAR) backscatter data (Leivens et al., 2021, 2022). The spicy-snow effort, a precursor to the crunchy-snow project, 1) implemented the Lievens et al. (2022) retrieval technique as an open-source software package and 2) evaluated algorithm performance using airborne lidar snow depth data. We found that there was promising snow depth information in the Sentinel-1 data, but the algorithm was not able to accurately capture snow depth. 

![spicy_vs_lidar](../../imgs/spicy_vs_lidar.png)

### How can Sentinel-1 SAR data be used to measure snow depth? 

"SAR sensors emit electromagnetic energy in the microwave range (1–40 GHz) and measure the amplitude and phase of the backscattered (returning) wave. Depending on the radar wavelength, microwaves can be transmissible through snow, but when the wavelength is within an order of magnitude of the diameter of snow grains (∼0.1–5 mm), volumetric scattering occurs. Additionally, surface scattering will occur within the snowpack at layer interfaces. (Naderpour et al., 2022; Tsang et al., 2022). Hence, for SAR frequencies approximately between 5–40 GHz, the presence of snow increases volumetric scattering relative to non-snow conditions (Fig. 1). Some SAR-based methods exploit this increased volumetric backscatter to retrieve measurements of snow depth and SWE (Tsang et al., 2022). These approaches are ineffective in wet snow conditions, where liquid water within the snowpack absorbs substantial microwave energy, leading to marked reductions in backscatter and limiting the depth of microwave penetration. 

The relationship between C-band volume scattering and snow depth is an ongoing area of investigation... A study using artificial snow in a laboratory setting showed a cross-polarized (orthogonal transmitting and receiving antennas) backscatter increase of 5 dB with a 1 m snow depth increase (Kendra, 1995). Two recent tower-based studies showed 2–5 dB increases in co-polarized backscatter for C-band radiation (Naderpour et al., 2022) and significant volume scattering from C-band cross-polarized backscatter at snowpack layering interfaces (Brangers et al., 2023), likely due to surface roughness effects. More recently, the development of dense media radiative transfer (DMRT) models has suggested that anisotropic clusters of snow grains may produce more crosspolarized backscatter from within the snowpack volume at C-band than previous isotropic scattering models suggested (West, 2000; Ding et al., 2010; Chang et al., 2014; Zhu et al., 2023). The increase in cross-polarized backscatter from these clusters may be sufficiently large to allow for measurements of snow depth changes as new snow increases the cross-polarized energy that is backscattered toward the sensor." -- Hoppinen et al. (2024)

![fig1](../../imgs/schematic.png)

### Why use a machine learning model to predict snow depth?
During the [`spicy-snow`](https://github.com/SnowEx/spicy-snow) effort, we found that the Lievens et al. (2022) algorithm was underparameterized, and not capable of capturing the full complexity of interactions between radar signals, snow, the ground, vegetation, and terrain features. Given the complexity of these interactions, and our poor physical understanding of them, we think a machine learning model with many parameters may be a more appropriate tool to predict snow depth than a simple statistical or physical algorithm. 

Besides potentially improved accuracy, an additional benefit of using a machine learning model is that we can easily include multiple datasets that contain direct or contextual information about snow depth. 

### What kind of machine learning model should we use?
This is up for discussion! Ideally, we want a model that:
- implicitly makes use of the spatial structure of the data
- is suitable for regression tasks (predicting snow depth)
- is possible to effectively train with the quantity of data we have
- is complex enough to capture the complexity of relationships in our data

Given these constraints, the first model we reached for was a [U-Net](https://en.wikipedia.org/wiki/U-Net) [convolutional neural network (CNN)](https://en.wikipedia.org/wiki/Convolutional_neural_network), which is a proven tool for performing regression tasks with image data. The specific architecture that works best is something we have to figure out ourselves! And other types of models may work even better, so it's worth experimenting. 

### How can we set up this machine learning problem?
In order to perform supervised training of a machine learning model, some "truth" values are required to compare against model predictions and update the model. We therefore require some very accurate snow depth data to compare against our model predictions. Luckily, the [Airborne Snow Observatory (ASO)](https://www.airbornesnowobservatories.com/) has collected and released a significant amount of lidar snow depth data over the Western U.S. since 2016, which is typically accurate to a few centimeters (Hoppinen et al., 2024). We can set up a machine learning problem where we compare our predicted snow depths to ASO lidar snow depth data, which will allow us to train a snow depth prediction model for the Western U.S. 

## Dataset preparation
This part of the notebook is intended to explain the method used to prepare the training, validation, and testing dataset.

### Data selection
Our first task was to download and clean all Airborne Snow Observatory 50-m resolution snow depth rasters, which will serve as our target dataset (or "label") in training, and allow us to evaluate model performance in validation and testing. These data need to be cleaned such that only valid/realistic values remained, since artifacts and spurious values might hamper the training process. We then needed to find all Sentinel-1 and Sentinel-2 data that overlapped spatially and temporally with the ASO data. Where a given ASO collection was nearby in time to a low-cloud Sentinel-2 collection and a morning Sentinel-1 collection, we also downloaded a summer Sentinel-1 collection to provide a snow-off comparison. We also downloaded matching elevation and forest cover fraction data. 

### Tiling and subsetting
To separate our training data from our testing and validation data, we defined a 25x25 km grid covering the entire ASO dataset. 80% of tiles in this grid were assigned to be training tiles, 10% validation, and 10% testing. To collect 128x128 pixel (6.4x6.4 km) image stacks that can serve as input data for a CNN, we randomly extracted subsets for each ASO collection (and associated remote sensing data) within a given tile. From the initial 259 ASO collections, the result is 30,749 training images, 2750 validation images, and 3592 testing images. 

### Normalization
In order to effectively train a CNN, we first need to normalize our input data to get everything on the same scale, which generally improves model convergence for various reasons. We will normalize each of our inputs between 0 and 1 before passing them to the model. There are multiple ways to accomplish this. Because we are performing a regression task, we want to preserve differences in values between images--therefore we don't want to normalize using the min and max or mean and standard deviation of each individual image. We instead want to normalize using min and max values across the entire dataset. However, we also need to consider the range of possible values for future use of this model. E.g., we don't want to normalize latitude assuming a maximum value of 49 degrees if we someday hope to apply our model in Alaska. We have already calculated or manually set the values for normalization for each input data type. 

## More information/resources
Snow depth retrieval from Sentinel-1:
- the [spicy-snow paper](https://egusphere.copernicus.org/preprints/2024/egusphere-2024-1018/egusphere-2024-1018.pdf)
- the [spicy-snow tutorial](https://github.com/SnowEx/spicy-snow/tree/main/contrib/brencher/tutorial)
- the [Lievens et al. (2022) paper](https://tc.copernicus.org/articles/16/159/2022/)

Machine learning:
- [A Recipe for Training Neural Networks](https://karpathy.github.io/2019/04/25/recipe/)
- [Pytorch docs](https://pytorch.org/tutorials/)